In [1]:
// подключаем необходимые библиотеки
import $ivy.`org.scalanlp::breeze:1.0`
import scala.collection.mutable.ArrayBuffer
import breeze.linalg._

import $ivy.$                         

import scala.collection.mutable.ArrayBuffer

import breeze.linalg._

In [2]:
// читаем файл с данными
val dataFile = io.Source.fromFile("Dataset.csv")

// подготавливаем матрицу
var dataMatrix = DenseMatrix.zeros[Double](0,28)

var i = 0
// Пробегаемся по каждай строке файла
for (line <- dataFile.getLines.drop(1)) { 
    
    // готовим массив для дальнейшего его перегона в матрицу 
    var dataArray = Array[Double]()
    
    // делим строку на элементы и обрезаем пробелы
    for (elem <- line.split(",")map(_.trim)){
        // если значения нет, то добавляем 0
        if(elem == "") {
            dataArray = dataArray :+ 0.0 
        // иначе добавляем в массив значение елемента, приведенного к целому числу
        }else{
            dataArray = dataArray :+ elem.toDouble
        }
    }
    
    // наполняем матрицу
    dataMatrix = DenseMatrix.vertcat(dataMatrix, DenseMatrix(DenseVector(dataArray)))
    i += 1
}



dataFile: io.BufferedSource = empty iterator
dataMatrix: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (28 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   1.0   1.0    37.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    23.0  ...
634995.0  0.0  463.0  1.0  3.0   0.0   3.0   3.0   -3.0   40.0  ...
634995.0  0.0  463.0  1.0  3.0   0.0   3.0   2.0   -3.0   54.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    29.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   36.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    11.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    62.0  ...
634995.0  0.0  463.0  1.0  77.0  46.0  31.0  74.0  15.0   31.0  ...
634995.0  0.0  463.0  1.0  6.0   0.0   1.0   6.0   -1.0   54.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    71.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    61.0  ...
634995.0  0.0  463.0  1.0  4.0   4.0   0.0   4.0   4.0    13.0  ...
634995.0  0.0  463.0  1.0  29.0  3.0   11.0  25.0  -8.0   49.0  ...
634995.0  0.0  463.0  1.0  3.0   1.0   2.0   3.0   -1.0   35.0  ...
634995.0  0.0  463.0  1.0  2.0   2.0   0.0   2.0   2.0    11.0  ...
634995.0  0.0  463.0  1.0  2.0   0.0   2.0   2.0   -2.0   37.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    0.0   ...
634995.0  0.0  463.0  1.0  2.0   2.0   0.0   2.0   2.0    11.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    43.0  ...
634995.0  0.0  463.0  1.0  2.0   2.0   0.0   2.0   2.0    18.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    64.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    51.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    67.0  ...
634995.0  0.0  463.0  1.0  8.0   8.0   0.0   8.0   8.0    12.0  ...
...
i: Int = 16

In [3]:
// разбиваем данные
val X = dataMatrix(0 to dataMatrix.rows - 1, 0 to dataMatrix.cols-2)
val Y = dataMatrix(::,27)

X: DenseMatrix[Double] = 634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    65.0  ... (27 total)
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    10.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    14.0  ...
634995.0  0.0  463.0  1.0  7.0   0.0   3.0   7.0   -3.0   62.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    58.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    60.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    68.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   1.0   1.0   -1.0   32.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    35.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    48.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    52.0  ...
634995.0  0.0  463.0  1.0  1.0   0.0   0.0   1.0   0.0    69.0  ...
634995.0  0.0  463.0  1.0  0.0   0.0   0.0   0.0   0.0    3.0   ...
634995.0  0.0  463.0  1.0  1.0   1.0   0.0   1.0   1.0    37.0  ...
634995.0  0.

import $ivy.$                                      

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._



In [0]:
/*
// применяем регрессионные модели
import breeze.stats.regression.{leastSquares, lasso}
val squaresModel = leastSquares(X, Y)
val lassoModel = lasso(X, Y, 0.01)
*/